# Evaluation Part I
我們將在這個範例介紹發展提示訊息的流程：  
  
提示訊息第一個版本的開發  
    -> 測試評估  
    -> 發生問題  
    -> 第二個版本的修正  
    -> 確認問題解決  
    -> 確認其他部分依然適用  
    
    到最後的自動化驗證。

### 環境設定
以下是環境的準備， 執行前請先在 `.env` 檔案內填入您自己的 openai api key。

In [1]:
import os
import openai
import sys
sys.path.append('../..')
import utils
import json
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo-16k", temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

### 幾個重要的 helper functions

helper function 1: 取得各個產品類別以及類別下的產品清單

In [3]:
products_and_category = utils.get_products_and_category()
products_and_category

{'電腦與筆記型電腦': ['TechPro Ultrabook',
  'BlueWave 電競筆電',
  'PowerLite 二合一電腦',
  'TechPro 桌機',
  'BlueWave Chromebook'],
 '手機和配件': ['SmartX ProPhone',
  'MobiTech 行動充電王',
  'SmartX MiniPhone',
  'MobiTech 無線充電器',
  'SmartX 耳機'],
 '電視和家庭劇院系統': ['CineView 4K 液晶電視',
  'SoundMax 家庭劇院',
  'CineView 8K 液晶電視',
  'SoundMax 音箱',
  'CineView OLED 電視'],
 '遊戲機和配件': ['GameSphere X',
  'ProGamer 搖桿',
  'GameSphere Y',
  'ProGamer 賽車方向盤',
  'GameSphere VR 頭盔'],
 '音響設備': ['AudioPhonic 降噪耳機',
  'WaveSound 藍牙喇叭',
  'AudioPhonic 真無線耳機',
  'WaveSound 音箱',
  'AudioPhonic 黑膠唱盤機'],
 '相機和攝影機': ['FotoSnap 單眼相機',
  'ActionCam 4K',
  'FotoSnap 無反光鏡相機',
  'ZoomMaster 攝影機',
  'FotoSnap 拍立得相機']}

helper function 2（版本一）: 由使用者的訊息取得詢問的產品以及類別資訊

In [4]:
def find_category_and_product_v1(user_input,products_and_category):

    delimiter = "####"
    system_message = f"""
    您將會收到客戶服務的查詢。
    該客戶服務查詢將以 {delimiter} 字符進行分隔。
    請輸出一個 Python 列表，其中每個物件都是一個 JSON 物件，每個物件具有以下格式：
    'category': <以下其中一種：電腦和筆記本，智慧手機和配件，電視和家庭影院系統，遊戲機和配件，音響設備，相機和攝像機>
    和
    'products': <必須在下面允許的產品中找到的產品列表內>

    類別和產品必須在客戶服務查詢中找到。
    如果提到了產品，它必須與下面允許的產品列表中的正確類別相關聯。
    如果找不到產品或類別，則輸出一個空列表。

    根據產品名稱和產品類別與客戶服務查詢的相關程度，列出所有相關的產品。
    不要從產品的名稱推測任何特性或屬性，如相對品質或價格。

    允許的產品以 JSON 格式提供。
    每個項目的鍵代表類別。
    每個項目的值是該類別中的產品列表。
    允許的產品：{products_and_category}

    
    """
    
    few_shot_user_1 = """我想要你們最貴的電腦。"""
    few_shot_assistant_1 = """ 
    [{'category': '電腦與筆記型電腦', \
    'products': ['TechPro Ultrabook', 'BlueWave 電競筆電', 'PowerLite 二合一電腦', 'TechPro 桌機', 'BlueWave Chromebook']}]
    """
    
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_1 },
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)


helper function 2（版本一） 的幾個測試評估查詢

In [5]:
customer_msg_0 = f"""如果我有預算限制的話我可以購買哪種電視？"""

products_by_category_0 = find_category_and_product_v1(customer_msg_0,
                                                      products_and_category)

print(products_by_category_0)

 
    [{'category': '電視和家庭劇院系統', 'products': ['CineView 4K 液晶電視', 'SoundMax 家庭劇院', 'CineView 8K 液晶電視', 'SoundMax 音箱', 'CineView OLED 電視']}]


In [6]:
customer_msg_1 = f"""我需要幫我的手機買個充電器"""

products_by_category_1 = find_category_and_product_v1(customer_msg_1,
                                                      products_and_category)

print(products_by_category_1)

 
    [{'category': '手機和配件', 'products': ['MobiTech 行動充電王', 'MobiTech 無線充電器']}]


In [7]:
customer_msg_2 = f"""
你有什麼電腦？"""

products_by_category_2 = find_category_and_product_v1(customer_msg_2,
                                                      products_and_category)
print(products_by_category_2)


    [{'category': '電腦與筆記型電腦', 'products': ['TechPro Ultrabook', 'BlueWave 電競筆電', 'PowerLite 二合一電腦', 'TechPro 桌機', 'BlueWave Chromebook']}]


In [8]:
customer_msg_3 = f"""
告訴我有關SmartX ProPhone和FotoSnap相機的資訊，
就是那款數位單眼相機的。另外你們有什麼電視？"""

products_by_category_3 = find_category_and_product_v1(customer_msg_3,
                                                      products_and_category)
print(products_by_category_3)


    [{'category': '手機和配件', 'products': ['SmartX ProPhone']}, {'category': '相機和攝影機', 'products': ['FotoSnap 單眼相機']}, {'category': '電視和家庭劇院系統', 'products': ['CineView 4K 液晶電視', 'SoundMax 家庭劇院', 'CineView 8K 液晶電視', 'SoundMax 音箱', 'CineView OLED 電視']}]


    較困難的測試項目

In [9]:
customer_msg_4 = f"""
跟我說明一下 CineView 電視，8K 的那一款，還有 Gamesphere 遊戲機，X 一款。
我有預算的限制，你們有什麼電腦可以選擇？
"""

products_by_category_4 = find_category_and_product_v1(customer_msg_4,
                                                      products_and_category)

# 預期只回覆 python 物件列表, 但是這個回覆有問題。
print(products_by_category_4)

 
    [{'category': '電視和家庭劇院系統', 'products': ['CineView 8K 液晶電視']}, {'category': '遊戲機和配件', 'products': ['GameSphere X']}]
    
    對於您的預算限制，我們有以下電腦選擇：
    [{'category': '電腦與筆記型電腦', 'products': ['TechPro Ultrabook', 'BlueWave 電競筆電', 'PowerLite 二合一電腦', 'TechPro 桌機', 'BlueWave Chromebook']}]


### 提示的修正

In [5]:
def find_category_and_product_v2(user_input,products_and_category):
    """
    增加： 不要輸出任何不是 JSON 格式的額外文字。

    以下的 few shot 的修正，中文的提示加了反而不正常。故省略。
    增加： 第二個例子（使用少量提示），用戶要求最便宜的電腦。 
    兩個少量提示的例子中，顯示的回應也都是 JSON 格式的完整產品列表。
    """
    delimiter = "####"
    system_message = f"""
    您將會收到客戶服務的查詢。
    該客戶服務查詢將以 {delimiter} 字符進行分隔。
    請輸出一個 Python 列表，其中每個物件都是一個 JSON 物件，每個物件具有以下格式：
    'category': <以下其中一種：電腦和筆記本，智能手機和配件，電視和家庭影院系統，遊戲機和配件，音響設備，相機和攝像機>
    和
    'products': <必須在下面允許的產品中找到的產品列表內>
    
    請不要輸出任何不是 JSON 格式的額外文字。
    輸出要求的 JSON 之後，請不要寫任何解釋性的文字。

    類別和產品必須在客戶服務查詢中找到。
    如果提到了產品，它必須與下面允許的產品列表中的正確類別相關聯。
    如果找不到產品或類別，則輸出一個空列表。

    根據產品名稱和產品類別與客戶服務查詢的相關程度，列出所有相關的產品。
    不要從產品的名稱推測任何特性或屬性，如相對品質或價格。

    允許的產品以 JSON 格式提供。
    每個項目的鍵代表類別。
    每個項目的值是該類別中的產品列表。
    允許的產品：{products_and_category}
    

    """
    
    few_shot_user_1 = """我想要最貴的電腦，你有什麼建議？"""
    few_shot_assistant_1 = """ 
    [{'category': '電腦與筆記型電腦', \
    'products': ['TechPro Ultrabook', 'BlueWave 電競筆電', 'PowerLite 二合一電腦', 'TechPro 桌機', 'BlueWave Chromebook']}]
    """
    
    # 以下的 few shot 的修正，中文的提示加了反而不正常。故省略。
    # few_shot_user_2 = """我想要最便宜的電腦，你有什麼建議？"""
    # few_shot_assistant_2 = """ 
    # [{'category': '電腦與筆記型電腦', \
    # 'products': ['TechPro Ultrabook', 'BlueWave 電競筆電', 'PowerLite 二合一電腦', 'TechPro 桌機', 'BlueWave Chromebook']}]
    # """
    
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_1 },
    #{'role':'user', 'content': f"{delimiter}{few_shot_user_2}{delimiter}"},  
    #{'role':'assistant', 'content': few_shot_assistant_2 },
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)

確認新版提示在有問題的訊息是不是正常了

test case 4 

In [11]:
customer_msg_4 = f"""
跟我說明一下 CineView 電視，8K 的那一款，還有 Gamesphere 遊戲機，X 一款。
我有預算的限制，你們有什麼電腦可以選擇？
"""

products_by_category_4 = find_category_and_product_v2(customer_msg_4,
                                                      products_and_category)

print(products_by_category_4)

 
    [{'category': '電視和家庭劇院系統', 'products': ['CineView 8K 液晶電視']}, {'category': '遊戲機和配件', 'products': ['GameSphere X']}, {'category': '電腦與筆記型電腦', 'products': ['TechPro Ultrabook', 'BlueWave 電競筆電', 'PowerLite 二合一電腦', 'TechPro 桌機', 'BlueWave Chromebook']}]


之前沒有問題的訊息也需要確認沒有受影響

In [12]:
customer_msg_0 = f"""如果我有預算限制的話我可以購買哪種電視？"""

products_by_category_0 = find_category_and_product_v2(customer_msg_0,
                                                      products_and_category)

print(products_by_category_0)

 
    [{'category': '電視和家庭劇院系統', 'products': ['CineView 4K 液晶電視', 'SoundMax 家庭劇院', 'CineView 8K 液晶電視', 'SoundMax 音箱', 'CineView OLED 電視']}]


### 自動化測試驗證的範例
先建立測試集

In [6]:
msg_ideal_pairs_set = [
    
    # eg 0
    {'customer_msg':"""如果我有預算限制的話我可以購買哪種電視？""",
     'ideal_answer':{
        '電視和家庭劇院系統':set(
            ['CineView 4K 液晶電視', 'SoundMax 家庭劇院', 'CineView 8K 液晶電視', 'SoundMax 音箱', 'CineView OLED 電視']
        )}
    },

    # eg 1
    {'customer_msg':"""我需要幫我的手機買個充電器""",
     'ideal_answer':{
        '手機和配件':set(
            ['MobiTech 行動充電王', 'MobiTech 無線充電器']
        )}
    },
    # eg 2
    {'customer_msg':f"""你有什麼電腦？""",
     'ideal_answer':{
           '電腦與筆記型電腦':set(
               ['TechPro Ultrabook', 'BlueWave 電競筆電', 'PowerLite 二合一電腦', 'TechPro 桌機', 'BlueWave Chromebook'
               ])
                }
    },

    # eg 3
    {'customer_msg':f"""告訴我有關SmartX ProPhone和FotoSnap相機的資訊，\
      就是那款數位單眼相機的。另外你們有什麼電視？""",
     'ideal_answer':{
        '手機和配件':set(
            ['SmartX ProPhone']),
        '相機和攝影機':set(
            ['FotoSnap 單眼相機']),
        '電視和家庭劇院系統':set(
            ['CineView 4K 液晶電視', 'SoundMax 家庭劇院', 'CineView 8K 液晶電視', 'SoundMax 音箱', 'CineView OLED 電視'])
        }
    }, 
    
    # eg 4
    {'customer_msg':"""跟我說明一下 CineView 電視，8K 的那一款，還有 Gamesphere 遊戲機，X 一款。
      我有預算的限制，你們有什麼電腦可以選擇？""",
     'ideal_answer':{
        '電視和家庭劇院系統':set(
            ['CineView 8K 液晶電視']),
        '遊戲機和配件':set(
            ['GameSphere X']),
        '電腦與筆記型電腦':set(
            ['TechPro Ultrabook', 'BlueWave 電競筆電', 'PowerLite 二合一電腦', 'TechPro 桌機', 'BlueWave Chromebook'])
        }
    },
    
    # eg 5
    {'customer_msg':f"""你有哪些智慧手機？""",
     'ideal_answer':{
        '手機和配件':set(
            ['SmartX ProPhone', 'MobiTech 行動充電王', 'SmartX MiniPhone', 'MobiTech 無線充電器', 'SmartX 耳機'
            ])
        }
    },
    # eg 6
    {'customer_msg':f"""如果我有預算限制.  你可以推薦一些智慧手機給我嗎？""",
     'ideal_answer':{
        '手機和配件':set(
            ['SmartX 耳機', 'SmartX MiniPhone', 'MobiTech 行動充電王', 'SmartX ProPhone', 'MobiTech 無線充電器']
        )}
    },

    # eg 7 # this will output a subset of the ideal answer
    {'customer_msg':f"""如果我朋友打算參加電競比賽，有什麼遊戲機適合我們朋友的？""",
     'ideal_answer':{
        '遊戲機和配件':set([
            'GameSphere X',
            'ProGamer 搖桿',
            'GameSphere Y',
            'ProGamer 賽車方向盤',
            'GameSphere VR 頭盔'
     ])}
    },
    # eg 8
    {'customer_msg':f"""你有什麼東西適合送給我從事攝影的朋友當作禮物？""",
     'ideal_answer': {
        '相機和攝影機':set([
        'FotoSnap 單眼相機', 'ActionCam 4K', 'FotoSnap 無反光鏡相機', 'ZoomMaster 攝影機', 'FotoSnap 拍立得相機'
        ])}
    },
    
    # eg 9
    {'customer_msg':f"""我想要一台時光機浴缸。""",
     'ideal_answer': []
    }
    
]

helper function 3: 比較回應的產品類別以及產品清單的理想值的差異性

In [7]:
import json
def eval_response_with_ideal(response,
                              ideal,
                              debug=False):
    
    if debug:
        print("回應內容： ")
        print(response)
    
    # json.loads() 只能讀取雙引號字串，無法讀取單引號字串
    json_like_str = response.replace("'",'"')
    
    # 讀取為 dict list
    l_of_d = json.loads(json_like_str)
    
    # 如果兩者都是空值時
    if l_of_d == [] and ideal == []:
        return 1
    
    # 如果兩者只有其中一個是空值時，明顯有錯誤。
    elif l_of_d == [] or ideal == []:
        return 0
    
    correct = 0    
    
    if debug:
        print("l_of_d： ")
        print(l_of_d)
    for d in l_of_d:

        cat = d.get('category')
        prod_l = d.get('products')
        if cat and prod_l:
            # 轉換為 set 以便比較
            prod_set = set(prod_l)

            # 取得理想回覆中的產品類別
            ideal_cat = ideal.get(cat)
            if ideal_cat:
                # 轉換為 set 以便比較
                prod_set_ideal = set(ideal.get(cat))
            else:
                if debug:
                    print(f"無法在理想回覆中找到 {cat} 類別")
                    print(f"理想回覆: {ideal}")
                continue
                
            if debug:
                print("prod_set\n",prod_set)
                print()
                print("prod_set_ideal\n",prod_set_ideal)

            # 檢查實際回應是否為理想回覆的子集
            if prod_set == prod_set_ideal:
                # 如果實際回應是理想回覆的子集，則正確
                if debug:
                    print("正確")
                correct +=1
            else:
                # 如果實際回應不是理想回覆的子集，則檢查是否為理想回覆的超集
                print("不正確")
                print(f"prod_set: {prod_set}")
                print(f"prod_set_ideal: {prod_set_ideal}")
                if prod_set <= prod_set_ideal:
                    print("實際回應只是理想回覆的子集")
                elif prod_set >= prod_set_ideal:
                    print("實際回應是理想回覆的超集")

    # 計算正確率
    pc_correct = correct / len(l_of_d)
        
    return pc_correct

In [16]:
print(f'Customer message: {msg_ideal_pairs_set[7]["customer_msg"]}')
print(f'Ideal answer: {msg_ideal_pairs_set[7]["ideal_answer"]}')

Customer message: 如果我朋友打算參加電競比賽，有什麼遊戲機適合我們朋友的？
Ideal answer: {'遊戲機和配件': {'ProGamer 賽車方向盤', 'ProGamer 搖桿', 'GameSphere VR 頭盔', 'GameSphere Y', 'GameSphere X'}}


In [17]:
response = find_category_and_product_v2(msg_ideal_pairs_set[7]["customer_msg"],
                                         products_and_category)
print(f'Response: {response}')

eval_response_with_ideal(response,
                              msg_ideal_pairs_set[7]["ideal_answer"])

Response: 
    [{'category': '遊戲機和配件', 'products': ['GameSphere X', 'ProGamer 搖桿', 'GameSphere Y', 'ProGamer 賽車方向盤', 'GameSphere VR 頭盔']}]


1.0

執行所有的測試案例並且計算正確率

In [8]:
import time

# 請注意，這個在任一個 api 呼叫逾時時，就會失敗。
score_accum = 0
for i, pair in enumerate(msg_ideal_pairs_set):
    print(f"範例 {i}")
    
    customer_msg = pair['customer_msg']
    ideal = pair['ideal_answer']

    # 從 customer_msg 找出產品類別和產品名稱    
    # print("Customer message",customer_msg)
    # print("ideal:",ideal)
    response = find_category_and_product_v2(customer_msg,
                                                      products_and_category)

    
    # 計算正確率
    # print("products_by_category",products_by_category)
    score = eval_response_with_ideal(response, ideal, debug=False)
    print(f"{i}: {score}")
    score_accum += score

    # 等待 20 秒。免費版的 openapi 有每分鐘 3 次的呼叫限制，如果確定沒有問題，可以把這行註解掉。
    #print("等待 20 秒。\n")
    time.sleep(20)
    

# 計算所有範例的正確率
n_examples = len(msg_ideal_pairs_set)
fraction_correct = score_accum / n_examples
print(f"{n_examples} 個測試案例的正確率 : {fraction_correct}")

範例 0
0: 1.0
範例 1
1: 1.0
範例 2
2: 1.0
範例 3
3: 1.0
範例 4
4: 1.0
範例 5
5: 1.0
範例 6
6: 1.0
範例 7
7: 1.0
範例 8
8: 1.0
範例 9
9: 1
10 個測試案例的正確率 : 1.0
